#### Build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

- Find the feature in the dataset that produces the best score for predicting 'crop".
- From this information, create a variable called best_predictive_feature, which
should be a dictionary containing the best predictive feature name as a key and the evaluation score (for the metric you chose) as the value.

In [1]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from IPython.display import clear_output
from sklearn.compose import ColumnTransformer

# Configure
random_seed = 202503

# Define a dict for feature's meanings
feature_meaning = {
'N': 'Nitrogen content ratio in the soil',
'P': 'Phosphorous content ratio in the soil',
'K': 'Potassium content ratio in the soil',
'pH': 'pH value of the soil', 
}

# Load the dataset
crops = pd.read_csv('data/soil_measures.csv')
print(crops.head())


    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice


In [2]:
# Inspect
print(crops.info())
print(crops.describe())
print(crops['crop'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None
                 N            P            K           ph
count  2200.000000  2200.000000  2200.000000  2200.000000
mean     50.551818    53.362727    48.149091     6.469480
std      36.917334    32.985883    50.647931     0.773938
min       0.000000     5.000000     5.000000     3.504752
25%      21.000000    28.000000    20.000000     5.971693
50%      37.000000    51.000000    32.000000     6.425045
75%      84.250000    68.000000    49.000000     6.923643
max     140.000000   145.000000   205.000000     9.935091
crop
rice           100
maize          100
ju

In [3]:
# Split data
X = crops.drop(columns='crop')
y = crops['crop']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=random_seed)

# Scale
scaler = StandardScaler()
all_cols = X.columns.to_list()
preprocessing = ColumnTransformer(transformers = [('scaler', scaler, all_cols)], remainder='drop')
transformed_array_train = preprocessing.fit_transform(X_train)
X_train_processed = pd.DataFrame(transformed_array_train, columns=preprocessing.get_feature_names_out())
transformed_array_test = preprocessing.transform(X_test)
X_test_processed = pd.DataFrame(transformed_array_test, columns=preprocessing.get_feature_names_out())

In [ ]:
# Perform parameter tunning and cross validation to decide the overall best model
# Instanciate estimators
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000000, random_state=random_seed)
rf = RandomForestClassifier(random_state=random_seed)
gbc = GradientBoostingClassifier(random_state=random_seed)

# Set up parameter grids
models = {
    logreg: {'C': np.exp(np.linspace(np.log(0.001), np.log(1000), 1000))},
    rf: {'n_estimators':[int(n) for n in np.linspace(40, 200, 160)]},
    gbc: {'learning_rate':np.linspace(0.001, 1, 1000), 
            'n_estimators':[int(n) for n in np.linspace(40, 200, 160)]},
            }
search_res = {}

# Iterate over all models and parameter grids, search for the best parameters for each model
for model, params in models.items():
    search = RandomizedSearchCV(model, params, n_iter=50, return_train_score=False, random_state=random_seed)
    search.fit(X_train_processed, y_train)

    # If return_train_score=True, search.cv_results_['mean_test_score'] 
    # gets an array of the mean_cv_scores

    search_res[model] = {
        # The best mean cv score out of all parameter combinitions
        'best_mean_cv_score': search.best_score_,
        # The best model fitted with the whole fitted data
        'best_model': search.best_estimator_, 
        'best_params': search.best_params_,
                         }
print(search_res)

{LogisticRegression(max_iter=1000000, multi_class='multinomial',
                   random_state=202503): {'best_mean_cv_score': 0.6896969696969697, 'best_model': LogisticRegression(C=801.5006961565414, max_iter=1000000,
                   multi_class='multinomial', random_state=202503), 'best_params': {'C': 801.5006961565414}}, RandomForestClassifier(random_state=202503): {'best_mean_cv_score': 0.7812121212121211, 'best_model': RandomForestClassifier(n_estimators=49, random_state=202503), 'best_params': {'n_estimators': 49}}, GradientBoostingClassifier(random_state=202503): {'best_mean_cv_score': 0.7854545454545454, 'best_model': GradientBoostingClassifier(learning_rate=0.027000000000000003, n_estimators=145,
                           random_state=202503), 'best_params': {'n_estimators': 145, 'learning_rate': 0.027000000000000003}}}


In [5]:
# Get the best cv score among the best tunned models
best_score = max([x['best_mean_cv_score'] for x in search_res.values()])
# Get the final best model according to the best cv score
get_model_by_score = lambda dt, v: dt['best_model'] if dt['best_mean_cv_score'] == v else None
best_model = [get_model_by_score(dt, best_score) for dt in search_res.values() if get_model_by_score(dt, best_score) is not None][0]
print(f'The best model is {best_model}')
# Get the test score of the best model
test_score = best_model.score(X_test_processed,y_test)

The best model is GradientBoostingClassifier(learning_rate=0.027000000000000003, n_estimators=145,
                           random_state=202503)


In [6]:
# Find the best preditive feature
importances = best_model.feature_importances_
idx_max = list(importances).index(max(importances))
best_feature = crops.columns[idx_max]
best_predictive_feature = {best_feature:test_score}

print(f'The best predictive feature is "{best_feature}, {feature_meaning[best_feature]}" \
with a score of {test_score:.4f}')

The best predictive feature is "K, Potassium content ratio in the soil" with a score of 0.8018
